In [10]:
import networkx as nx
from networkx.algorithms import community #  
import itertools
import matplotlib.pyplot as plt
import csv
from networkx.algorithms.community.quality import modularity
import random
from keybert import KeyBERT
import time
from gensim import models,corpora
from math import log, exp
from collections import Counter


In [11]:
#工具函数
#计算pmi
def GetPointwiseMutualValue(word_1,word_2,doclist):
    value_1=0
    value_2=0
    value_mutual=0
    count_word=1  #一共出现多少词,加一不影响结果，只是防止除0
    for item in doclist:
        item=item['data']
        if word_1 in item: 
            value_1+=1
        if word_2 in item: 
            value_2+=1
        if word_1 in item and word_2 in item: 
            value_mutual+=1
            #print(word_1)
            #print(word_2)
            #print(item)
        count_word=count_word+item.count(' ')+1 #用空格作为分词的依据
    #value_1= value_1/count_word
    #value_2= value_2/count_word  
    #value_mutual=value_mutual/count_word
    if value_mutual != 0:
        #value_pmi=log((value_mutual/(value_1*value_2)),2)   #原始pmi
        value_pmi=value_mutual/(max(value_1,value_2))
    else:
        value_pmi=0 
    return value_pmi    

In [12]:
#工具函数  过滤事件
def filteEvent(event_final):
    global filter_word
    event_return=[]
    for item in event_final:
        flag=0
        #print('开始过滤')
        #print(item['word'])
        for word in item['word']:
            #print(word)
            if (word  in filter_word):
                flag=1
        if(flag==0):
            event_return.append(item)
    return event_return                

In [13]:
#主要函数  1
#从文件夹中读取type信息
def getTypeWordAndData(file,datelist):
    word_list=[]
    with open(file+'-node_type.csv',encoding="utf-8") as f:
     f_csv = csv.reader(f)
     next(f_csv)
     for row in f_csv:
        dic={}
        dic['word']=row[1]
        dic['size']=eval(row[2])
        dic['type']=row[3]
        #if(row[1] not in ['chatgpt', 'gpt', 'ai', 'chat', 'openai', 'artificialintelligence', 'chatbot', 'google', 'bing', 'https']):
        word_list.append(dic)
    doclist=[]
    with open('../推特GPT/GPT.csv',encoding="utf-8") as f:
         f_csv = csv.reader(f)
         pos = 1
         for row in f_csv:
            if  len(row)>=6:  #控制聚类文本数目并处理缺失数据,全部跑太多了跑不了
                #print(row[0][:10])  #取到日期
                if row[0][:10] in datelist: #判断时间
                    #print(row[0])
                    dic={}
                    dic['like_count']=eval(row[4])
                    dic['retweet_count']=eval(row[5])
                    dic['data'] = row[2]
                    doclist.append(dic)
                    pos+=1  
    return word_list,doclist

In [14]:
#主要函数  2   V1
#从全部词中取得有联系的词
def getPMIAndEventWord_v1(word_list,doclist):
    derail_value = max(word_list, key=lambda d: d["type"])["type"]
    event_word=[]
    #print(word_list)
    for i in range(len(word_list)):
        for j in range(i+1,len(word_list)):
            #同话题，不考虑离群
            if(word_list[i]['type']==word_list[j]['type'] and word_list[i]['type']!=derail_value):
                value_pmi=GetPointwiseMutualValue(word_list[i]['word'],word_list[j]['word'],doclist)
                if(value_pmi>=0.05):    #话题增益
                    dic={}
                    dic['word']=[]
                    dic['word'].append(word_list[i]['word'])
                    dic['word'].append(word_list[j]['word'])
                    dic['type']=word_list[i]['type']
                    dic['pmi']=value_pmi
                    dic['flag']=0
                    dic['keyword'] = max([word_list[i], word_list[j]], key=lambda x: x['size'])
                    event_word.append(dic)
    event_word = sorted(event_word, key=lambda d: d["pmi"],reverse=True)
    event_final=[]
    #为每个
    #存在相同的词则合并
    remove_word=['ai','https','msft']
    #for i in range(len(event_word)):
    #    dic={}
    #    #print(event_word[i])
    #    #print(event_word[i]['flag'])
    #    if(event_word[i]['flag']==0):  #表示没有被操作过
    #        for j in range(i+1,len(event_word)):
    #            for j_word in event_word[j]['word']:
    #                if(j_word in event_word[i]['word']):
    #                    event_word[j]['flag']=1
    #            if(event_word[j]['flag']==1):
    #                event_word[i]['word']=event_word[i]['word']+event_word[j]['word']          
    #                event_word[i]['word']=list(set(event_word[i]['word']))
    #        event_word[i]['word'] = [x for x in event_word[i]['word'] if x not in remove_word]        
    #        event_final.append(event_word[i])
    #        #去除过于核心的词和没有意义的词
    for i in range(len(event_word)):
        dic={}
        if(event_word[i]['flag']==0):  #表示没有被操作过
            for j in range(i+1,len(event_word)):
                if(event_word[i]['keyword']==event_word[j]['keyword']):
                    event_word[j]['flag']=1  #标为可合并
            if(event_word[j]['flag']==1):   
                event_word[i]['word']=event_word[i]['word']+event_word[j]['word']          
                event_word[i]['word']=list(set(event_word[i]['word']))
            event_word[i]['word'] = [x for x in event_word[i]['word'] if x not in remove_word]        
            event_final.append(event_word[i])     
    #print(event_final)        
    event_word = enlargeWord(event_final,doclist)
    return event_word        

In [15]:
#主要函数  2  
#从全部词中取得有联系的词
def getPMIAndEventWord(word_list,doclist):
    derail_value = max(word_list, key=lambda d: d["type"])["type"]
    event_word=[]
    #print(word_list)
    for i in range(len(word_list)):
        for j in range(i+1,len(word_list)):
            #同话题，不考虑离群
            if(word_list[i]['type']==word_list[j]['type'] and word_list[i]['type']!=derail_value):
                value_pmi=GetPointwiseMutualValue(word_list[i]['word'],word_list[j]['word'],doclist)
                if(value_pmi>=0.1):    #话题增益
                    keyword = max([word_list[i], word_list[j]], key=lambda x: x['size'])
                    if(keyword not in event_word):
                        event_word.append(keyword)  #存核心词,然后再看第二步
    event_final=[]
    for keyword in event_word:
        dic={}
        dic['keyword']=keyword
        #print(f'核心词{keyword}')
        dic['word']=[]
        dic['word'].append(keyword['word'])
        dic['flag']=0
        for word in word_list:
            if(word['word'] != keyword['word']):
                pmi = GetPointwiseMutualValue(keyword['word'],word['word'],doclist)
                if(pmi>=0.05 ):  #看pmi值
                    if(keyword['type'] == word['type'] or word['type'] == derail_value):   #存一个type的 
                        dic['word'].append(word['word'])
        event_final.append(dic)                 
    event_final = enlargeWord(event_final,doclist)
    
    #处理可能存在的重复问题。  如果一个事件的keyword在另一个事件的word中，则合并
    event_final_empty=[]
    for i in range(len(event_final)):
        #print(event_final[i])
        for j in range(i+1,len(event_final)):
            if(event_final[i]['keyword']['word'] in event_final[j]['word'] or event_final[j]['keyword']['word'] in event_final[i]['word']):   #判断keyword在word列表中的情况
                event_final[i]['word']=list(set(event_final[i]['word']+event_final[j]['word']))
                event_final[j]['flag']=1
        if(event_final[i]['flag']==0):
           event_final_empty.append(event_final[i])
    event_final = event_final_empty
    #过滤event
    event_final = filteEvent(event_final)                
    return event_final        

In [16]:
#主要函数  3
#选择相关文档
def enlargeWord(event_word,doclist):
    global proportion
    for item in event_word:
      word_list=item['word']
      doc_list=[]
      #print(word_list)
      for doc in doclist:
          text=doc['data']
          text = [word for word in text.split() if word.strip('#') != "" or word.strip('@') != ""]  # 以空格为单位分词 
          count=0   #统计词数
          for single_word in text:
              if(single_word in word_list): 
                  count+=1
          if(count/len(word_list)>=proportion):  #占比超过多少的视为相关文档,初始化设置为0.3
              #if(doc['like_count']+doc['retweet_count']*3>=5): #过滤无用户行为的
                    doc_list.append(doc)
      item['doc']= doc_list
    return  event_word             
                    

In [17]:
file_list=[
    '../推特GPT/成图csvV3/1月6日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/1月9日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/1月12日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/1月15日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/1月18日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/1月21日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/1月24日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/1月27日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/1月30日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月2日起11728全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月5日起22806全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月8日起24368全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月11日起17814全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月14日起18673全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月17日起16115全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月20日起17954全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月23日起15076全数据，边为相似度,门限0.8',
    '../推特GPT/成图csvV3/2月26日起13956全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月1日起17547全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月4日起13555全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月7日起15353全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月10日起12912全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月13日起25126全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月16日起25263全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月19日起20431全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月22日起24489全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月25日起19776全数据，边为相似度,门限0.8',
    #'../推特GPT/成图csvV3/3月28日起16429全数据，边为相似度,门限0.8',
]
date_list=[
    ['2023-01-06','2023-01-07','2023-01-08'],
    ['2023-01-09','2023-01-10','2023-01-11'],
    ['2023-01-12','2023-01-13','2023-01-14'],
    ['2023-01-15','2023-01-16','2023-01-17'],
    ['2023-01-18','2023-01-19','2023-01-20'],
    ['2023-01-21','2023-01-22','2023-01-23'],
    ['2023-01-24','2023-01-25','2023-01-26'],
    ['2023-01-27','2023-01-28','2023-01-29'],
    ['2023-01-30','2023-01-31','2023-02-01'],
    ['2023-02-02','2023-02-03','2023-02-04'],
    ['2023-02-05','2023-02-06','2023-02-07'],
    ['2023-02-08','2023-02-09','2023-02-10'],
    ['2023-02-11','2023-02-12','2023-02-13'],
    ['2023-02-14','2023-02-15','2023-02-16'],
    ['2023-02-17','2023-02-18','2023-02-19'],
    ['2023-02-20','2023-02-21','2023-02-22'],
    ['2023-02-23','2023-02-24','2023-02-25'],
    ['2023-02-26','2023-02-27','2023-02-28'],
    #['2023-03-01','2023-03-02','2023-03-03'],
    #['2023-03-04','2023-03-05','2023-03-06'],
    #['2023-03-07','2023-03-08','2023-03-09'],
    #['2023-03-10','2023-03-11','2023-03-12'],
    #['2023-03-13','2023-03-14','2023-03-15'],
    #['2023-03-16','2023-03-17','2023-03-18'],
    #['2023-03-19','2023-03-20','2023-03-21'],
    #['2023-03-22','2023-03-23','2023-03-24'],
    #['2023-03-25','2023-03-26','2023-03-27'],
    #['2023-03-28','2023-03-29','2023-03-30'],
]
proportion = 0.4  #事件中有多少个词出现在一条文本中，则将该文本视为该事件的文本。  该值为门限比例，有超过这个比例的词出现则视为True     

#从全数据中得到需要去除的词
filter_word=['chatgpt', 'gpt', 'ai', 'chat', 'openai', 'artificialintelligence', 'chatbot', 'google', 'bing', 'https','data','language']

                    

In [ ]:
#全时间段   MyModule最终结果评估
召回率 = 22/27
正确率 = 76/94    #预测出的词有多少对了
#混淆率 = 5/94     #预测出的词有多少是其他事件的,这个不是很好
事件召回率 = 10/11
事件正确率 = 36/39      #预测出的事件有不少于40%词属于某一个确定事件


In [18]:
for i in range(len(file_list)):
    print(date_list[i])
    word_list,doclist = getTypeWordAndData(file_list[i],date_list[i])
    word_list_now=word_list.copy()  #用以暂存这次的,注意深复制
    if(i != 0):
        #word_list=word_list+word_list_pre
        #将上一次的词加入本次word
        for word_pre in word_list_pre:
            flag=0
            for item in word_list:
                if(item['word']==word_pre['word']):
                    flag=1
            if(flag==0):
                word_list.append(word_pre)        
    event_get = getPMIAndEventWord(word_list,doclist)  #得到的事件
    for item in event_get:
        
        #for doc in item['doc']:
        #    print(doc['data'])
        if(len(item['doc'])>5):   #认为检测出的事件显著
            print(f"新事件:{item['word']}")
            print(f"共有文档:{len(item['doc'])}")
            #for doc in item['doc'][:50]:  #只取前20个
            #    print(doc)
    word_list_pre = word_list_now    

['2023-01-06', '2023-01-07', '2023-01-08']
新事件:['education', 'teachers', 'writing', 'schools', 'essay', 'students']
共有文档:65
新事件:['tweet', 'tweets']
共有文档:130
['2023-01-09', '2023-01-10', '2023-01-11']
新事件:['writing', 'essay', 'content']
共有文档:83
新事件:['students', 'teachers', 'education']
共有文档:33
新事件:['essays', 'essay']
共有文档:116
['2023-01-12', '2023-01-13', '2023-01-14']
新事件:['essays', 'poem', 'essay']
共有文档:13
新事件:['tweet', 'tweets']
共有文档:117
['2023-01-15', '2023-01-16', '2023-01-17']
新事件:['students', 'teachers', 'education']
共有文档:44
新事件:['essay', 'essays']
共有文档:102
['2023-01-18', '2023-01-19', '2023-01-20']
新事件:['prompts', 'prompt']
共有文档:216
['2023-01-21', '2023-01-22', '2023-01-23']
新事件:['bitcoin', 'crypto']
共有文档:32
新事件:['students', 'teachers', 'essay']
共有文档:32
新事件:['cybersecurity', 'malware']
共有文档:25
新事件:['prompts', 'prompt']
共有文档:198
新事件:['tweet', 'tweets']
共有文档:123
['2023-01-24', '2023-01-25', '2023-01-26']
新事件:['exam', 'exams']
共有文档:235
新事件:['marketing', 'content', 'business']
共有文档:7